In [ ]:
import numpy as np
import pandas as pd
import random

# --- Settings ---
num_agents = 100
num_weeks = 50
bar_threshold = 65  # global crowd threshold for Average Thinkers
memory_decay_weights_3 = [0.6, 0.3, 0.1]
memory_decay_weights_2 = [0.7, 0.3]

# --- Initialize agents ---
agents = []

for agent_id in range(num_agents):
    strategy = random.choice(['Optimist', 'Pessimist', 'Average'])
    memory_length = 3 if strategy in ['Optimist', 'Pessimist'] else 2
    agent = {
        'Agent_ID': agent_id,
        'Strategy': strategy,
        'Risk_Tolerance': random.randint(60, 75),
        'Memory_Length': memory_length,
        'Belief': 55  # initial Bayesian prior
    }
    agents.append(agent)

# --- Initialize dataset and history ---
records = []
attendance_history = []

# --- Start Simulation ---
for week in range(1, num_weeks + 1):
    decisions = {}

    # Each agent predicts and decides
    for agent in agents:
        agent_id = agent['Agent_ID']
        strategy = agent['Strategy']
        risk_tolerance = agent['Risk_Tolerance']
        memory_length = agent['Memory_Length']
        belief = agent['Belief']

        # Step 1: Get memory-based attendance average
        if len(attendance_history) >= memory_length:
            memory = attendance_history[-memory_length:]
            weights = memory_decay_weights_3 if memory_length == 3 else memory_decay_weights_2
            weighted_avg = sum(w * a for w, a in zip(weights, memory)) / sum(weights)
        else:
            weighted_avg = belief  # use belief if not enough memory

        # Step 2: Decision based on strategy and rules
        if strategy == 'Optimist':
            if len(attendance_history) < memory_length:
                decision = 'Go' if risk_tolerance >= 65 else 'Stay'
            else:
                if risk_tolerance >= 65 and weighted_avg < risk_tolerance:
                    decision = 'Stay'
                else:
                    decision = 'Go'
        elif strategy == 'Pessimist':
            if len(attendance_history) < memory_length:
                decision = 'Stay' if risk_tolerance >= 65 else 'Go'
            else:
                if risk_tolerance >= 65 and weighted_avg < risk_tolerance:
                    decision = 'Go'
                else:
                    decision = 'Stay'
        elif strategy == 'Average':
            if len(attendance_history) < memory_length:
                decision = 'Go'
            else:
                decision = 'Go' if weighted_avg < bar_threshold else 'Stay'

        decisions[agent_id] = decision

    # Step 3: Calculate actual attendance
    actual_attendance = sum(1 for d in decisions.values() if d == 'Go')
    attendance_history.append(actual_attendance)

    # Step 4: Update beliefs
    for agent in agents:
        agent['Belief'] = (agent['Belief'] + actual_attendance) / 2

    # Step 5: Save records
    for agent_id, decision in decisions.items():
        agent = next(a for a in agents if a['Agent_ID'] == agent_id)
        record = {
            'Week': week,
            'Agent_ID': agent_id,
            'Strategy': agent['Strategy'],
            'Risk_Tolerance': agent['Risk_Tolerance'],
            'Memory_Length': agent['Memory_Length'],
            'Decision': decision,
            'Actual_Attendance': actual_attendance
        }
        records.append(record)

# Create DataFrame
df = pd.DataFrame(records)



,Week,Agent_ID,Strategy,Risk_Tolerance,Memory_Length,Decision,Actual_Attendance
0,1,0,Average,61,2,Go,72
1,1,1,Average,66,2,Go,72
2,1,2,Optimist,71,3,Go,72
3,1,3,Pessimist,72,3,Stay,72
4,1,4,Pessimist,63,3,Go,72


In [ ]:
# Save the DataFrame to CSV
csv_path = "../data/el_farol_simulation.csv"
df.to_csv(csv_path, index=False)

In [ ]:
# Reload the CSV to confirm content
df_loaded = pd.read_csv(csv_path)
df_loaded.head()